In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json

# Kafka consumer configuration
KAFKA_TOPIC = "your_topic_name"
KAFKA_BROKERS = "localhost:9092"

# Spark session creation (replace with your configuration)
spark = SparkSession.builder \
    .appName("KafkaToIcebergStreaming") \
    .config("spark.sql.warehouse", "spark-warehouse") \
    .getOrCreate()

# Read streaming data from Kafka as JSON
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKERS) \
    .option("subscribe", KAFKA_TOPIC) \
    .load() \
    .select(from_json(col("value").cast("string"), schema="your_schema_json"))  # Replace with your schema

# Process the data (modify this based on your needs)
processed_df = kafka_df.withColumn("processed_column", col("original_column") * 2)

# Write the processed data to Iceberg table in Minio (replace with your table details)
processed_df.write \
    .format("iceberg") \
    .option("catalog-name", "my_iceberg_catalog").option("table-name", "my_iceberg_table").option("warehouse", "s3://warehouse/my_iceberg_table").mode("append").saveAsTable("my_iceberg_table")  # Temporary table for streaming

# Start the streaming query
query = processed_df.writeStream \
    .outputMode("append") \
    .trigger(processingTime="5 seconds").start()

# Wait for the streaming query to terminate
query.awaitTermination()

# Stop the Spark session
spark.stop()


24/05/29 11:25:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.